In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib notebook

In [2]:
data = np.load("train_data.npy")
labels = np.load("train_labels.npy")
test_data = np.load("test_data.npy")
labels = labels.astype(int)

### Prepairing Data

#### Test Data

In [3]:
test_data_count, test_width, test_height =  test_data.shape

In [4]:
test_data = test_data.flatten().reshape(test_data_count, test_width * test_height)

In [5]:
test_data = test_data/test_data.max()

#### Train Data

In [6]:
data_count, width, height =  data.shape

In [7]:
train_data = data.flatten().reshape(data_count, width * height)

In [8]:
train_data = train_data / train_data.max()

In [9]:
labels[labels == 0] = -1

In [10]:
x_train, x_test, y_train, y_test = train_test_split(train_data, labels, test_size=0.25, random_state=0)

### Train

In [11]:
def sigmoid(x, b):
    """
    :param x: 1D-array, features vector for one sample
    :param b: 1D-array, betas of the model
    :return: sigmoid (logistic) of (x, b)
    """
    return 1 / (1 + np.exp(-x.dot(b)))

def grad(y, x, b):
    """
    Gradient of log-loss for one sample
    :param y: label of the sample (1 or -1)
    :param x: 1D-array, features vector for the sample
    :param b: 1D-array, betas of the model
    """
    return y*x*(np.exp(-y * x.dot(b))/(1 + np.exp(-y * x.dot(b))))

def loss(X, Y, b):
        h = sigmoid(X, b)
        return (-Y * np.log(h) - (1 - Y) * np.log(1 - h)).mean()
    
def train(Y, X, steps=10, lr=0.001):
    """
    Get optimal betas
    :param Y: 1D-array, labels of all data like [1,1,-1,1,-1,-1,1,...]
    :param X: 2D-array of shape (features number, samples number), i.e. vector of all feature vectors
    :return: 1D-array of size of features number, bettas
    """
    b = np.zeros(X.shape[1])
    for s in range(steps):
        for i in range(len(X)):
            x = X[i]
            y = Y[i]
            gradient = grad(y, x, b)
            b += lr * gradient
        if s%10 == 1:
            print("step: {}, loss: {}".format(s, loss(X, Y, b)))
    return b

In [12]:
betas = train(y_train, x_train, steps=100)

step: 1, loss: -2.5746794247506823
step: 11, loss: -3.6768084648758412
step: 21, loss: -4.038886906056058
step: 31, loss: -4.261778094058302
step: 41, loss: -4.423053142176824
step: 51, loss: -4.548926913801499
step: 61, loss: -4.651657715183862
step: 71, loss: -4.738123683694463
step: 81, loss: -4.812652741280719
step: 91, loss: -4.878180274441726


In [13]:
predictions = sigmoid(x_test, betas)

In [14]:
predictions[predictions >= 0.5] = 1
predictions[predictions < 0.5] = -1

In [15]:
(predictions == y_test).mean() * 100

99.8

In [16]:
test_pred = sigmoid(test_data, betas)

In [17]:
test_pred[test_pred >= 0.5] = 1
test_pred[test_pred < 0.5] = 0

In [18]:
# test_pred[7]

In [19]:
# plt.imshow(test_data[7].reshape((28,28)))

In [20]:
np.save("karen_ghandilyan.npy", test_pred)

### Testing on sklearn

In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
print ('Accuracy from scratch: {0}'.format((predictions == y_test).sum().astype(float) / len(predictions)))
print ('Accuracy from sk-learn: {0}'.format(lr.score(x_test, y_test)))

Accuracy from scratch: 0.998
Accuracy from sk-learn: 0.998


In [24]:
pred = lr.predict(test_data)

In [25]:
pred[pred == -1] = 0

In [26]:
(test_pred == pred).mean() * 100

99.93569131832798